# 02 - LU Decomposition

---

### 1. Introduction

**LU Decomposition** is a method of factoring a square matrix $A$ into the product of two other matrices: a **lower triangular matrix ($L$)** and an **upper triangular matrix ($U$)**.

$$ A = L \cdot U $$

The structure of these matrices is typically:
- **$L$ (Lower Triangular)**: Has ones on its main diagonal and zeros above it.
- **$U$ (Upper Triangular)**: Has the pivots from the elimination process on its main diagonal and zeros below it.

For example:
$$ 
A = 
\begin{pmatrix}
2 & 3 \\
4 & 7
\end{pmatrix}
= 
\begin{pmatrix}
1 & 0 \\
2 & 1
\end{pmatrix}
\begin{pmatrix}
2 & 3 \\
0 & 1
\end{pmatrix}
= L \cdot U
$$

This decomposition is one of the most important techniques in numerical linear algebra. Its main power lies in efficiently solving linear systems.

### 2. Solving Linear Systems with LU Decomposition

The primary motivation for finding the LU decomposition of a matrix $A$ is to solve the system $Ax = b$ more efficiently.

The process involves two simple steps:
1.  Substitute $A$ with $LU$: The original equation $Ax = b$ becomes $LUx = b$.
2.  Introduce an intermediate vector $y$ where $Ux = y$.

This breaks the single, difficult problem into two easy-to-solve triangular systems:

**Step 1: Solve $Ly = b$ for $y$ using Forward Substitution.**
$$ 
\begin{pmatrix}
1 & 0 & 0 \\
l_{21} & 1 & 0 \\
l_{31} & l_{32} & 1
\end{pmatrix}
\begin{pmatrix} y_1 \\ y_2 \\ y_3 \end{pmatrix} = 
\begin{pmatrix} b_1 \\ b_2 \\ b_3 \end{pmatrix}
$$

**Step 2: Solve $Ux = y$ for $x$ using Back Substitution.**
$$ 
\begin{pmatrix}
u_{11} & u_{12} & u_{13} \\
0 & u_{22} & u_{23} \\
0 & 0 & u_{33}
\end{pmatrix}
\begin{pmatrix} x_1 \\ x_2 \\ x_3 \end{pmatrix} = 
\begin{pmatrix} y_1 \\ y_2 \\ y_3 \end{pmatrix}
$$

#### Why is this better?
The expensive part of solving a system is the elimination phase (which has a complexity of $O(n^3)$). Finding the LU decomposition is essentially performing this elimination phase on matrix $A$. Once this is done, solving for $x$ using forward and back substitution is very fast ($O(n^2)$).

This is incredibly useful when you need to solve the same system $Ax=b$ for **many different vectors $b$**. You perform the expensive decomposition only once, and then rapidly solve for each $b$.

### 3. Finding L and U with Gaussian Elimination

The LU decomposition is a direct byproduct of the Gaussian Elimination process.

- The **U matrix** is simply the **upper triangular matrix** that results from the forward elimination phase.
- The **L matrix** is constructed from the **multipliers** used during the elimination. The entry $l_{ij}$ (for $i > j$) is precisely the multiplier used to zero out the element $a_{ij}$.

Let's find the LU decomposition for the matrix:
$$ A = 
\begin{pmatrix}
2 & 1 & 1 \\
4 & -6 & 0 \\
-2 & 7 & 2
\end{pmatrix}
$$

#### Step 1: Eliminate the first column
The pivot is $a_{11} = 2$.
- Multiplier for row 2: $m_{21} = \frac{4}{2} = 2$. So, $l_{21} = 2$. New row 2 is $L_2 - 2L_1$.
- Multiplier for row 3: $m_{31} = \frac{-2}{2} = -1$. So, $l_{31} = -1$. New row 3 is $L_3 - (-1)L_1 = L_3+L_1$.
$$ A^{(1)} = 
\begin{pmatrix}
2 & 1 & 1 \\
0 & -8 & -2 \\
0 & 8 & 3
\end{pmatrix}
$$

#### Step 2: Eliminate the second column
The pivot is $a_{22} = -8$.
- Multiplier for row 3: $m_{32} = \frac{8}{-8} = -1$. So, $l_{32} = -1$. New row 3 is $L_3 - (-1)L_2 = L_3+L_2$.
$$ U = A^{(2)} = 
\begin{pmatrix}
2 & 1 & 1 \\
0 & -8 & -2 \\
0 & 0 & 1
\end{pmatrix}
$$

#### Resulting Matrices
$$ L = 
\begin{pmatrix}
1 & 0 & 0 \\
l_{21} & 1 & 0 \\
l_{31} & l_{32} & 1
\end{pmatrix}
= 
\begin{pmatrix}
1 & 0 & 0 \\
2 & 1 & 0 \\
-1 & -1 & 1
\end{pmatrix}
, \quad
U = 
\begin{pmatrix}
2 & 1 & 1 \\
0 & -8 & -2 \\
0 & 0 & 1
\end{pmatrix}
$$

### 4. The Need for Pivoting and the PA=LU Decomposition

Just like in standard Gaussian Elimination, the algorithm described above fails if a pivot element is zero. It also suffers from numerical instability if a pivot is very small. Therefore, in any practical implementation, we **must use pivoting**.

When we swap rows during elimination, we are effectively reordering the equations of our system. This is equivalent to multiplying our original matrix $A$ by a **Permutation Matrix ($P$)**. A permutation matrix is just an identity matrix with its rows reordered.

Therefore, the decomposition of a general square matrix is not $A=LU$, but rather:
$$ PA = LU $$

Where $P$ is the permutation matrix that records all the row swaps performed during pivoting.

#### Solving with $PA=LU$
The solving process is slightly modified:
1. Start with $Ax = b$.
2. Multiply both sides by $P$: $PAx = Pb$. (This applies the row swaps to the vector $b$ as well).
3. Substitute $PA$ with $LU$: $LUx = Pb$.
4. Let $y = Ux$, which leads to the two familiar triangular solves:
   - Solve $Ly = Pb$ for $y$.
   - Solve $Ux = y$ for $x$.

### 5. Python Implementation

Libraries like NumPy and SciPy have highly optimized functions for LU decomposition. `scipy.linalg.lu` is the standard, as it also returns the permutation matrix $P$.

In [2]:
import numpy as np
from scipy.linalg import lu, solve_triangular

# Let's use the example from the manual calculation
A = np.array([[2., 1., 1.],
              [4., -6., 0.],
              [-2., 7., 2.]])

b = np.array([5., -2., 9.]) # A consistent b vector for this example

# Perform PA = LU decomposition
P, L, U = lu(A)

print("--- PA=LU Decomposition ---")
print("Permutation Matrix (P):\n", P)
print("\nLower Triangular Matrix (L):\n", L)
print("\nUpper Triangular Matrix (U):\n", U)

# Verify that P @ A is indeed equal to L @ U
print("\nVerification (P @ A):\n", P @ A)
print("\nVerification (L @ U):\n", L @ U)

def solve_lu(P, L, U, b):
    """Solves the system Ax=b using a pre-computed PA=LU decomposition."""
    # 1. Apply permutations to b -> Pb
    Pb = P @ b
    
    # 2. Solve Ly = Pb for y using forward substitution
    # solve_triangular is a highly optimized solver for triangular systems
    y = solve_triangular(L, Pb, lower=True)
    
    # 3. Solve Ux = y for x using back substitution
    x = solve_triangular(U, y, lower=False)
    
    return x

print("\n--- Solving the System ---")
solution = solve_lu(P, L, U, b)
print("Solution x:", solution)

# Verify with numpy's standard solver
print("NumPy's np.linalg.solve(A, b):", np.linalg.solve(A, b))

--- PA=LU Decomposition ---
Permutation Matrix (P):
 [[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]

Lower Triangular Matrix (L):
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [-0.5  1.   1. ]]

Upper Triangular Matrix (U):
 [[ 4. -6.  0.]
 [ 0.  4.  1.]
 [ 0.  0.  1.]]

Verification (P @ A):
 [[ 4. -6.  0.]
 [ 2.  1.  1.]
 [-2.  7.  2.]]

Verification (L @ U):
 [[ 4. -6.  0.]
 [ 2.  1.  1.]
 [-2.  7.  2.]]

--- Solving the System ---
Solution x: [1. 1. 2.]
NumPy's np.linalg.solve(A, b): [1. 1. 2.]


### 6. Ill-Conditioned Matrices

An **ill-conditioned** matrix is one that is very sensitive to small changes in its entries or in the vector $b$. Numerically, this often happens when a matrix is "close" to being singular.

When solving a system with an ill-conditioned matrix, even with stable methods like LU with pivoting, small floating-point errors can be magnified and lead to large errors in the final solution.

A classic example is the Hilbert matrix, where the columns are nearly linearly dependent.

The **condition number** of a matrix is a formal measure of this sensitivity. A small condition number (close to 1) means the matrix is well-conditioned. A very large condition number indicates that the matrix is ill-conditioned.

In [3]:
from scipy.linalg import hilbert

# A well-conditioned matrix (our example)
cond_A = np.linalg.cond(A)
print(f"Condition number of our example matrix A: {cond_A:.2f} (Well-conditioned)")

# An ill-conditioned matrix (Hilbert matrix)
H = hilbert(5)
cond_H = np.linalg.cond(H)
print(f"\nHilbert Matrix of size 5:\n", H)
print(f"\nCondition number of H: {cond_H:.2e} (Extremely ill-conditioned)")

Condition number of our example matrix A: 20.83 (Well-conditioned)

Hilbert Matrix of size 5:
 [[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]

Condition number of H: 4.77e+05 (Extremely ill-conditioned)


### 7. Key Takeaways: Summary of LU Decomposition

Here are the most important points to remember about LU Decomposition:

1. **Primary Benefit: Efficiency**

   * The main advantage of LU decomposition is its efficiency when solving ( Ax = b ) for **multiple different ( b ) vectors**.
   * The expensive part (factorizing ( A ), which is ( O(n^3) )) is done only **once**.
   * Each subsequent solution for a new ( b ) is very fast, requiring only one forward and one back substitution (both are ( O(n^2) )).

2. **How to Use (with SciPy):**

   * Always use a reliable library function like `scipy.linalg.lu`, which performs **pivoting automatically**.
   * The function returns three matrices: `P, L, U` such that `P @ A = L @ U`.
   * To solve for ( x ), you must apply the permutation matrix to ( b ) first (`Pb = P @ b`) before performing the two triangular solves.

3. **Matrix Condition is Crucial**

   * LU decomposition with pivoting is a **numerically stable algorithm**, but it cannot fix an inherently problematic matrix.
   * If a matrix is **ill-conditioned** (has a very large condition number), it means the problem itself is sensitive to small changes.
   * In such cases, even small floating-point errors can be amplified, leading to an inaccurate solution, regardless of the algorithm used.